# LightGBM

In [2]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


ModuleNotFoundError: No module named 'lightgbm'

## Lendo o Dataset tratado, transformando-o em um DataFrame e armazenando nas variáveis, as colunas referentes as suas características e rótulos

In [ ]:
df = pd.read_csv('dataset_tratado.csv')

x = df.drop('ZSN', axis=1)
y = df['ZSN']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

train_data = lgb.Dataset(x_train, label=y_train)
test_data = lgb.Dataset(x_test, label=y_test, reference=train_data)

In [ ]:
params = {
    'objective': 'binary',  
    'metric': 'binary_error',  
    'boosting_type': 'gbdt',
    'num_leaves': 35,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

In [ ]:
num_round = 100
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

y_pred = bst.predict(x_test, num_iteration=bst.best_iteration)
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

print(classification_report(y_test, y_pred_binary, zero_division=0))